# Project Introduction

__[Department of Transportation's Responsibilities](https://www.google.com/search?client=safari&rls=en&q=department+of+transportation+responsibilities&ie=UTF-8&oe=UTF-8)__

Through this dataset, I have identified the three following patterns:
1. IDEA 1 w/ LOCAL LINK
2. IDEA 2 w/ LOCAL LINK
3. IDEA 3 w/ LOCAL LINK

## Resources:

- __[Kaggle-Full](https://www.kaggle.com/datasets/sobhanmoosavi/us-accidents/data)__

- __[Kaggle-Sampled](https://drive.google.com/file/d/1U3u8QYzLjnEaSurtZfSAS_oh9AT2Mn8X/edit)__

- __[Bing API](https://learn.microsoft.com/en-us/bingmaps/rest-services/traffic/get-traffic-incidents#supported-http-methods)__

- __[MQuest API](https://developer.mapquest.com/documentation/api/traffic/incidents/get.html)__

- __['A Countrywide Traffic Accident Dataset'](https://arxiv.org/pdf/1906.05409)__

- __['Accident Risk Prediction based on Heterogenous Sparse Data: New Dataset & Insights](https://arxiv.org/pdf/1909.09638)__

- __['Census Data'](https://www.census.gov/acs/www/data/data-tables-and-tools/subject-tables/)__

# 1-Setup Environment

## Libraries

In [ ]:
#Utilities
import warnings
import os

# Data Basics
import pandas as pd
import numpy as np
import missingno as msgno

#Time Data
from datetime import date, timedelta


#PySpark
from pyspark.sql import SparkSession
from pyspark.mllib.stat import Statistics
import pyspark.sql.functions as F

#Visuals
import matplotlib.pyplot as plt
import seaborn as sns


# Statistical Analysis
from scipy.stats import zscore, f_oneway, chi2_contingency, chisquare
import statsmodels.api as sm
from statsmodels.formula.api import ols
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from yellowbrick.cluster import KElbowVisualizer, SilhouetteVisualizer
from sklearn.preprocessing import StandardScaler, OneHotEncoder


# Spatial Tools
import geopy
import geopy.distance
from geopy.geocoders import Nominatim
import census
import requests

## Info on New Libraries
For improved analysis, these libraries were included, but not covered in the course material:

- __[Geopy:  ](https://geopy.readthedocs.io/en/stable/)__

- Census:
    * __[PyPi](https://pypi.org/project/census/)__
    * __[API](https://www.census.gov/data/developers/guidance/api-user-guide.html)__

- MissingNo:  
    * __[Library](https://github.com/ResidentMario/missingno)__
    
    * __[Tutorial](https://www.geeksforgeeks.org/python-visualize-missing-values-nan-values-using-missingno-library/)__

- __[Yellowbrick](https://www.scikit-yb.org/en/latest/)__

## Settings

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
pd.set_option('display.width', 500)
_LITE_SWITCH_ = False
_SPARK_ = False

## Custom Functions

### For Database Creation

In [ ]:
def census_data(yr,state_county=False):
    base_link = f'https://api.census.gov/data/{yr}/acs/acs1?get=group(B08301)&ucgid=pseudo(0100000US$0400000)'
    #base_link = f'https://api.census.gov/data/{yr}/acs/acs5/subject?'
    if state_county:
        add_on = f'get=group(B08301)&POPGROUP=001&ucgid=pseudo(0100000US$0500000)'
    else:
        add_on = f''
        #add_on = f'get=group(B08301)&ucgid=pseudo(0100000US$0400000)'

    census_link = base_link + add_on
    
    r = requests.get(census_link)
    df = pd.DataFrame(r.json())
    df.columns = df.iloc[0]
    df = df.drop(0)
    df = df.set_index('NAME')
    columns = [c for c in df.columns if c.endswith('E')]
    df = df[columns]
    col_names = ['Total','Car_Truck_Van',
                'DriveAlone',
                'Carpool','2Person_Pool','3Person_Pool','4Person_Pool','56Person_Pool','7UpPerson_Pool',
                'Public','Bus','Subway','LongDistanceRail','LightRail','Ferry',
                'Taxi','Motorcycle','Bicycle','Walked','Other','WorkFromHome'
                ]

    df.columns = col_names
    df = df.astype(int)
    df[col_names] = df[col_names].apply(lambda r:r/r['Total'],axis=1)
    columns_filtered = ['DriveAlone','Carpool','Bus','Subway','LongDistanceRail','LightRail',
                        'Ferry','Taxi','Motorcycle','Bicycle','Walked','Other','WorkFromHome']
    return df[columns_filtered]

### Exploration Functions

In [ ]:
# For High-level data exploration
def count_outliers(df_col,cap=3):
    zs = zscore(df_col)
    return df_col[zs > cap].shape[0]

### Date-Related Feature Engineering

In [ ]:
# Holiday Indicators
# Given by Google:
def calculate_memorial_day(year):
    # Get the last day of May
    last_day_of_may = date(year, 5, 31)

    # Get the weekday of the last day of May (0=Monday, 6=Sunday)
    weekday = last_day_of_may.weekday()

    # Calculate the offset to get to the last Monday
    offset = (weekday - 0) % 7

    # Calculate Memorial Day
    memorial_day = last_day_of_may - timedelta(days=offset)

    return memorial_day

# Given by Google:
def get_thanksgiving_date(year):
    # Get the first day of November
    first_november = date(year, 11, 1)
    
    # Calculate the day of the week for the first of November (0 = Monday, 6 = Sunday)
    first_day_weekday = first_november.weekday()
    
    # Calculate the number of days to add to get to the first Thursday
    days_to_first_thursday = (3 - first_day_weekday) % 7
    
    # Calculate the date of the first Thursday of November
    first_thursday = first_november + timedelta(days=days_to_first_thursday)
    
    # Add 3 weeks (21 days) to get to the fourth Thursday (Thanksgiving)
    thanksgiving_date = first_thursday + timedelta(days=21)
    
    return thanksgiving_date

def get_labor_day(year):
    first = date(year,9,1)
    d = first
    while d.weekday() > 0:
       d = d + timedelta(days=1)
    return d

### Functionality for Running Statistical Analysis & Reporting

In [ ]:
def test_category(feature_name,dataset):
    # Time of Impact--ANOVA
    model_time = ols(f"Q('Time_of_Impact(hr)') ~ C({feature_name})", data=dataset).fit()
    anova_table_time = sm.stats.anova_lm(model_time, typ=2)
    # Distance of Impact--ANOVA
    model_distance = ols(f"Q('Distance(mi)') ~ C({feature_name})", data=dataset).fit()
    anova_table_distance = sm.stats.anova_lm(model_distance, typ=2)
    # Severity Distribution--ANOVA
    contingency_table = pd.crosstab(dataset[feature_name], dataset['Severity'])
    chi_results = chi2_contingency(contingency_table)
    return anova_table_time, anova_table_distance, chi_results

def display_categories(feature_name,dataset):
    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    #sns.boxplot(x=feature_name, y='Severity', data=dataset, palette='viridis')
    sns.boxplot(hue=feature_name, y='Severity', data=dataset, palette='viridis')
    plt.title('Accident Severity by '+feature_name)
    plt.subplot(1, 2, 2)
    dataset[feature_name].value_counts().plot(kind='bar', color='skyblue')
    plt.title('Total Accidents by '+feature_name)
    plt.tight_layout()
    plt.show()
    pass

In [ ]:
def test_results(feature_name,dataset,alpha=0.99,visuals=True,reporting=True):
    anova_time,anova_distance,chi_test= test_category(feature_name,dataset)
    p_vals = dict()
    p_vals['time'] = ('ANOVA',anova_time['PR(>F)'].iloc[0])
    p_vals['distance'] = ('ANOVA', anova_distance['PR(>F)'].iloc[0])
    p_vals['severity'] = ('CHI-Squared', chi_test[1])

    test_results = []
    for topic, p_val in p_vals.items():
        msg = f"Based on the {p_val[0]} test's p-value of {p_val[1]:.3f}, "
        null_hypo = f"the null hypothesis that there is no significant difference in incident {topic}"
        null_hypo += f" across different {feature_name}s."
        if (1 - alpha) > p_val[1]:
            msg += f"we reject " + null_hypo
        else:
            msg += f"we fail to reject " + null_hypo
        if(reporting): print(msg)
        test_results.append(msg)
    if visuals:
        display_categories(feature_name=feature_name,dataset=dataset)
    return test_results, [anova_time,anova_distance,chi_test]



## Load Dataset(s)

### Main Dataset
Will depend on settings: lite_switch gives option to use the scaled down dataset.  _SPARK_ will give option of loading with pyspark (which will also determine functionality through out project).

In [ ]:
if _SPARK_:
    spark = SparkSession.builder.appName("Accident Data Project").getOrCreate()
    data = spark.read.csv('US_Accidents_March23.csv',header=True,inferSchema=True)
else:
    if _LITE_SWITCH_:
        data = pd.read_csv('US_Accidents_March23_sampled_500k.csv')
    else:
        data = pd.read_csv('US_Accidents_March23.csv')    

### Other User-Built Datasets

In [ ]:
commute_options = ['NAME','Year','DriveAlone','Carpool','Bus','Subway','LongDistanceRail','LightRail','Taxi','Motorcycle','Bicycle','Walked','Other','WorkFromHome']
all_commutes = pd.DataFrame(columns=commute_options)
for yr in range(2016,2024):
    if(os.path.exists(f'Commute_{yr}.csv')):
        commute_census = pd.read_csv(f'Commute_{yr}.csv')
    else:
        try:
            commute_census = census_data(yr)
        except:
            commute_census = census_data(yr-1)
        commute_census.to_csv(f'Commute_{yr}.csv')
    commute_census['Year'] = yr
    
    all_commutes = pd.concat([all_commutes,commute_census],axis=0)

    all_commutes

In [ ]:
locator = Nominatim(user_agent='Capstone-1')
cities = ['NYC','Chicago','Miami','Atlanta','Charlotte','Dallas','Houston','Denver','LA','Seattle']
city_locals = pd.DataFrame(index=cities,columns=['lat','long'])
for city in cities:
    location=locator.geocode(city)
    city_locals.loc[city,'lat'] = location.latitude
    city_locals.loc[city,'long'] = location.longitude
city_locals

# 2-Initial EDA

## Schema & Feature Basics

In [ ]:
if _SPARK_:
    data.printSchema()
    print("Features: ",len(data.columns))
    print("Entries:  ",data.count())
else:
    data.info()

In [ ]:
if _SPARK_:
    data.show(5,vertical=False)
else:
    data.head(5)

Very large dataset with over 7.7 million observations.  (I also chose to do some initial analysis ont he kaggle-provided sampled dataset which contains only 500k observations.)  So we will have to handle with care either by using Spark or through other tricks.  

Dimensionally much more managable with just 45 features.

1. Identifiers in ID and Source:  Some of the research treats the Source as an important consideration but for this project's purposes, I think it can be ignored.

2. Nature of Incident:
- 'Severity' would appear to be the primary variable of interest; 'Description' might be interesting to experiment with.
    * Upon further research, there is a 'Distance(mi)' feature that reflects the mileage of road that was impacted.
    * Furthermore, we can calculate the duration of impact by calculating the delta between 'Start_Time' & 'End_Time'
    * We may be able to also create a feature of impacted area, depending on what we find with 'End_Lat' and 'End_Lng'

- 'Description' feature might be an interesting avenue to explore.  However, I suspect that it is collected/generated after the fact so there wouldn't be much predictive value.

3. A number of location features
    * 'Start_Lat'/'Start_Lng' are easy enough to interpret; 'End_Lat' _ 'End_Lng' are there to define the space the accident affects.  However, we already have the 'Distance(mi)'.  

    * We have address features: 
        - City, ZipCode, County and State all have utility that is immediately benefecial.
        - Street may be interesting if we can use it to identify types of roads.
        - Airport Code may be an interesting way to look at flight traffic vs road incidents.  However, as discussed below, it is unclear what this feature actually represents.
        - We can likely get rid of Country feature.
    



- Next are 10 binary features which seem to provide some information about the road infrastructure at the location of the accident.  


4. We have a handful of time-related features: 
- 'Start_Time' and 'End_Time' need to be [converted to datetimes](#datetimes).  Right now they are just strings and there appears to be an issue with inconsistent formatting.

- It may be interesting to use these values to engineer some features like time of day, season, weekday/weekend, etc.

- The information contained in 'Weather_Timestamp' is unclear and I do not believe the Timezone data will be worthwhile to retain since the start/end time features are local (according to Kaggle).

- There are 5 binary features indicating whether or not it was light or dark out at the time of the incident.

5. There are 9 (not including 'Weather_Timestamp') features on the weather:
- Temp, Wind Chill, Humidity, Pressure, Visibility, Wind Speed, and Precipitation are all continuous features with pretty straight-forward explanations.  They may be useful in a regression analysis.

- Weather Condition may be preferable under these circumstances since the most important target feature will be the number of accidents under certain circumstances.  (As disussed above, we cannot create a probability metric without a massive amount of extra data resources.)

- HOWEVER, this feature space's cardinality is troubling large and there are major imbalances. So we would need to consolidate:
    * We could do this manually by choosing a handful of the most frequent categories and assigning the less frequent categories into one of them.
    * But this requires some discretion which may not be preferable.  Instead, we look at the option of clustering the observatins based on the numerical weather features. 

6. There are 13 binary variables which indicate the presence of a different infrastructure/roadway structure/feature.
- Six strucutres are obviously meant to decrease accident frequency (& severity?):
    * Stop
    * Traffic Signal
    * Roundabout
    * Turning Loop
    * Traffic Calming (?)
    * Bump

- For three, their presence would intuitively increase the likelihood of accidents:
    * Amenity
    * Railway
    * Station

- Without additional info, the other four's intuitive impact is unclear:
    * No Exit
    * Crossing
    * Junction
    * Give Way

- It may also be informative to understand the interactions of these categories:
    * Are some mutually exclusive?
    * Do combinations of occurences linearly affect the number of accidents? How about the nature of an accident? 

In [ ]:
# Organize features into global variables:
_FEATURES_ = data.columns.to_list()

In [ ]:


_NUMERICS_ = ['Severity','Distance(mi)','Temperature(F)',
              'Wind_Chill(F)','Humidity(%)','Pressure(in)',
              'Visibility(mi)','Wind_Speed(mph)','Precipitation(in)'
              ]

In [ ]:
_TARGET_ = ['Severity','Distance(mi)']

_INFRASTRUCTURE_ = ['Stop','Traffic_Signal','Roundabout','Turning_Loop',
                    'Traffic_Calming','Bump','Amenity','Railway',
                    'Station','No_Exit','Crossing','Junction',
                    'Give-Way'
                    ]

_WEATHER_ = ['Weather_Condition','Temperature(F)','Wind_Direction'
             'Wind_Chill(F)','Humidity(%)','Pressure(in)',
             'Visibility(mi)','Wind_Speed(mph)','Precipitation(in)'
            ]

In [ ]:
_USELESS_ = ['Source','Weather_Timestamp']

## Missing Values

In [ ]:
if _SPARK_:
    missing = data.select(*[F.sum(F.isnull(F.col(c)).cast("int")).alias(c) for c in data.columns])
    print(missing.show(vertical=True))
else:
    print(data.isna().sum().sort_values(ascending=False))

There is a lot of missing data.  But let's first point out the positives:
- The two primary target variables, Severity & Distance(mi), do not have any missing observations.  

- We also have no missing observations of time (Start and End) so we should be able to create the third feature, impact duration, without issue.

- With so few observations missing a description, it is hard to imagine just blindly dropping those would impact our analysis.

- Some observations are missing address features (Street, City + Zipcode) but again the number is a small fraction (even if there is no overlap) of the total database so blindly dropping would be unlikely to be a concern.  Furthermore, all the observations have county and state features. As a result, we have plenty of options which will depend on the topics we choose to dig deep on.

- This leaves us with only a few groups of features which have missing values that requries some consideration before handling:  
    * End longitude/latitude
    * Weather
    * Night/Day

But let's first look at how the co-occurence of these missing observations.

In [ ]:
msgno.heatmap(data);

Good to know:
* Confirmed that Latitude/Longitude Data are strictly missing together.

* Weather data seems to be missing together in general but not perfectly missing and not a uniform relationship (Precipitation and Wind Direction for example).

* The day/night variables are strictly missing together.  

### Ending Coordinates

The simplest solution would be to just setting missing end coordinates to the observation's start coordinates.  But is this a common occurence among the observations which none of these variables are missing?

In [ ]:
same_startVend = data[(data['Start_Lat']==data['End_Lat']) & (data['Start_Lng']==data['End_Lng'])]

same_startVend.shape[0]/data['End_Lat'].notnull().shape[0]

5% of the observations which have both start & end coordinates.  So it is not uncommon but not frequent enough to make me comfortable changing half of the observations.

We have speculated that there is a relationship between distance feature and the coordinates.  Let's explore that further to inform our decision to handle missing values. 

In [ ]:
(same_startVend['Distance(mi)'] > 0).sum()

So the distance metric for all of the observations with equal start and end coordinates is 0.

Let's consider the same analysis for those observations missing the end coordinates.

In [ ]:
missing_ends = data[data['End_Lat'].isnull()]['Distance(mi)']
missing_ends.describe()

In [ ]:
(missing_ends==0).sum() / missing_ends.shape[0]

So for a significant portion of the observations missing the end coordinates, the distance metric is also 0.  

What does distance for the rest look like?

In [ ]:
data[(data['End_Lat'].isnull()) & (data['Distance(mi)'] > 0)]['Distance(mi)'].describe()

For these observations (especially the large outliers), it does not appear advisable to force the end coordinates to equal the start.

ACTION DECISION:  Set all missing ending coordinates equal to their starting coordinates.  (Possibility that this feature may ultimately be tossed in the future.)

### Weather
Based on some research and intuition, we can make some decisions on how to handle missing values.

#### Conditions

In [ ]:
data.Weather_Condition.isnull().sum()/data.shape[0]

It's only 2% of the data and, as it stands, I have no way have extrapolating how any individual observation should be classified.  Therefore, it is probably safe to remove.

ACTION DECISION: Remove all observations missing the Weather Condition feature.

ACTION DECISION: Drop Missing Temperature.

#### Precipitation, Wind Speed & Wind Direction

In [ ]:
(data['Precipitation(in)']==0).sum()/data[data['Precipitation(in)'].notnull()].shape[0]

Most (> 90%) of the observations not missing precipitation have 0. So it doesn't seem unreasonable to set missing values to 0. (It may be worthwhile to check against the weather condition in the future.)

ACTION DECISION: Set missing observations of precipitation to 0.

In [ ]:
(data['Wind_Speed(mph)']==0).sum()/data[data['Wind_Speed(mph)'].notnull()].shape[0]

While not as frequent, there are plenty (13%) of observations with wind speed equal to 0.  So it doesn't seem unreasonable to set missing values to 0. (It may be worthwhile to check against the observations wind direction and/or wind chill in the future.)

ACTION DECISION: Set missing observations of wind speed to 0.

In [ ]:
data['Wind_Direction'].value_counts(normalize=True,dropna=False)

A significant portion of the observations have some equivalent of calm, which I interpet it as 0 wind.  So I do not believe it is unreasonable to assume missing values to be the equivalent to 0 wind as well.  (Again, it may be worthwhile to check against the other wind features in the future.)

ACTION DECISION: Set missing observtions of wind direction to 'CALM' or some equivalent.  (Will depend on how I end up cleaning the Wind_Direction feature.)

##### Wind Chill

In [ ]:
data[data['Wind_Chill(F)']==data['Temperature(F)']].shape[0]/data.shape[0]

Over half of the observations occur where there is no additional effect on temp from the wind, so I do not think it is too dangerous to set missing wind-chill values equal to the temperture. (It may be worthwhile to check against the wind speed in the future.)

ACTION DECISION: Set missing values of wind chill to the  observation's temperture value.

#### Pressure, Visibility & Humidity

Per this google search https://www.google.com/search?client=safari&rls=en&q=can+air+pressure+be+0&ie=UTF-8&oe=UTF-8, air pressure cannot be 0 (except in a vacuum).  So without any anchor value to use, I do not think it wise to assign missing observations to any value.  

ACTION DECISION: Remove all observations missing a value for the Pressure feature.

There is also no obvious value to use for visibility.

ACTION DECISION: Remove all observations missing a value for the Pressure feature.

In [ ]:
(data['Humidity(%)']==0).sum()/data[data['Humidity(%)'].notnull()].shape[0]

There are no observations with 0% humidity, so we do not have any reasonable anchor value to use once again.  

ACTION DECISION: Remove all observations missing a value for the Humidity feature.

## Duplicates

There are no duplicates to deal with:

In [ ]:
if _SPARK_:
    duplicates = int(data.count() - data.dropDuplicates().count())
    duplicates.show()
else:
    print(data.duplicated().sum())

Nothing to worry about.

## Outliers

In [ ]:
if _SPARK_:
    pass
else:
    print(pd.DataFrame({c:{z:count_outliers(data[c],z) for z in [3,5,10,15,20]} for c in _NUMERICS_}).T)

Distance seems to be the only feature in the original dataset (lite or full) that has significant outliers.

In [ ]:
print(f'{(data['Distance(mi)'] > 100).sum()} of the observations are greater than 100 miles.  Lets drop these')

ACTION DECISION: Drop observations with distance impacted to be over 100 miles.  

## High-Level Features

### Nature of Accident

We will eventually create another feature to describe the nature of any accident but for now, we have severity (a discrete variable) and distance.

In [ ]:
data['Severity'].value_counts(normalize=True)

We have significant class imbalances.  Even though it is unclear what methodology (if any) is used to determine severity, this imbalance makes sense:  Low-impact incidents are likely underreported and the highest-impact incidents are, fortunately, rare. 

In [ ]:
data['Distance(mi)'].value_counts(bins=25,normalize=True).sort_index(ascending=True)

We know from exploring this feature's missing values and outliers what the issues for this feature.  The binning makes it seem that there are negative values but this is not the case.

In [ ]:
msg = f'{(data['Distance(mi)']==0).sum()/data.shape[0]:.2f} of observations have 0 impact distance and '
msg += f'{(0 > data['Distance(mi)']).sum()} observations with negative impact distance.'

print(msg)

### Time Features

To explore properly, we need to convert to datetime type (from pandas).  Also, need to convert into seconds and then into hours.

It is required to set the 'format' parameter to "mixed".  (Too many observations to understand why / explore observations setting off the error.)


In [ ]:
start = pd.to_datetime(data['Start_Time'],format='mixed')
end = pd.to_datetime(data['End_Time'],format='mixed')
time_change = (end - start).astype('timedelta64[s]').dt.seconds/3600
time_change.describe()

In [ ]:
time_change.value_counts(bins=25,normalize=True).sort_index(ascending=True)

For the most part, this distribution makes sense.  Again despite the binning, there are no negative values for the time of impact.

In [ ]:
msg = f'{(time_change==0).sum()} of the observations have 0 impact duration and '
msg += f'{(0 > time_change).sum()} observations with negative impact duration.'

print(msg)

It is unusual to see that some observations have no duration but it such a small number and purposely possible that it means essentially 0.  So no need to stess.

### Address Features

In [ ]:
data['Country'].value_counts()

Confirming that all observations are in the USA so we can infact ignore this featue in the future.

ACTION DECISION: Remove column/feature as part of data cleaning.

In [ ]:
_USELESS_.extend(['Country'])

#### State

In [ ]:
# Check for bad strings
unclean = data['State'].str.startswith(' ').sum() + data['State'].str.endswith(' ').sum()
unclean

There are none to deal with.

In [ ]:
data['State'].value_counts(dropna=False,normalize=True)

California seems very high even adjusting for population and area size.  It appears we do not have data from the territories like Puerto Rico (PR), US Virgin Islands (VI) but we do have data from District of Columbia (DC).Comments & Actions.  Concerns about balance?

#### City

In [ ]:
# Same Double Check
unclean = data['City'].str.startswith(' ').sum() + data['City'].str.endswith(' ').sum()
unclean

There are no bad string to deal with.

In [ ]:
data['City'].value_counts(dropna=False,normalize=True).sort_values(ascending=False).head(25)

No concerns with balance (or over-representation).  Interesting to see some of these cities make it in the top 25.

#### Zipcode

In [ ]:
unclean = data['Zipcode'].str.startswith(' ').sum() + data['Zipcode'].str.endswith(' ').sum()
unclean

No issues.

In [ ]:
data['Zipcode'].value_counts(dropna=False,normalize=True).sort_values(ascending=False)

Like with city, there doesn't appear to be any concern with the balance.  The syntax with the 4-digit extension (used to tell delivery system a more specific location) should probably be removed since we want consistency.

ACTION DECISION: Force all zipcodes to follow simple five digit syntax.

In [ ]:
data['Zipcode'].str.contains(r'[0-9]{5}').sum()

#### Airport Code

In [ ]:
unclean = data['Airport_Code'].str.startswith(' ').sum() + data['Airport_Code'].str.endswith(' ').sum()
unclean

No issue.

In [ ]:
data['Airport_Code'].value_counts(dropna=False,normalize=True).sort_values(ascending=False)

The distribution itslef isn't necessarily a concern. However, these codes do not make much sense.  If KCQT is actually a code for an airport, it would be refering to Seattle (WA) and KRDU would be refering to the airport in Raleigh-Durham (NC).  The latter especialy would be surprising from common sense.  Where is LAX?

In [ ]:
lax_count = data[data['Airport_Code']=='LAX'].shape[0]
print(f'There are {lax_count} incidents with LAX as its airport code.')

Upon further review, this may be indication that this feature is not the traditional airport codes but weather stations!  If that is the case, this feature would have limited analytical value (for now).

ACTION DECISION: Remove this feature from the dataset.

In [ ]:
_USELESS_.extend(['Airport_Code'])

#### Timezone

In [ ]:
data['Timezone'].value_counts(normalize=True,dropna=False)

No information can be gleaned from this feature.  I will be grouping the states later on but in ways that either make sense for analytics or for "real-life" recommendations.

ACTION DECISION: Remove this feature from the dataset.

In [ ]:
_USELESS_.extend(['Timezone'])

#### Street Type

In [ ]:
unclean = data['Street'].str.startswith(' ').sum() + data['Street'].str.endswith(' ').sum()
unclean

Actually need to handle this if we are to use this feature and need to make sure it is handled in data processing stage.

In [ ]:
streets = data['Street'].str.strip()
unclean = streets.str.startswith(' ').sum() + streets.str.endswith(' ').sum()
unclean

Simple fix.

ACTION DECISION: Strip the string feature in data processing.

In [ ]:
streets.value_counts(dropna=False,normalize=True).sort_values(ascending=False).head(25)

Not surprising.  We may want to use this information in feature engineering below.

Need to figure out if there is a consistent syntax for street address.

We are going to use the following resources to consider if we can group these observations based on the street name where the accident is reported to have occured.

__[Google](https://www.google.com/search?q=us+highways+and+interstates&client=safari&sca_esv=a2d3658fd380a756&rls=en&ei=t65JaN2uMMKh5NoPo6mVkQo&oq=US+highways+vs%C2%A0&gs_lp=Egxnd3Mtd2l6LXNlcnAaAhgCIhBVUyBoaWdod2F5cyB2c8KgKgIIADILEAAYgAQYkQIYigUyBxAAGIAEGAoyBRAAGIAEMgYQABgWGB4yBhAAGBYYHjIGEAAYFhgeMgsQABiABBiGAxiKBTILEAAYgAQYhgMYigUyCxAAGIAEGIYDGIoFMggQABiABBiiBEiDTFDVDFjWP3AFeAGQAQCYAXOgAdcKqgEEMTYuMbgBAcgBAPgBAZgCFqACgwzCAgoQABiwAxjWBBhHwgIKEAAYgAQYQxiKBcICERAuGIAEGLEDGNEDGIMBGMcBwgILEAAYgAQYsQMYgwHCAgUQLhiABMICDhAuGIAEGLEDGIMBGNQCwgILEAAYgAQYsQMYiwPCAggQABiABBiLA8ICCBAAGIAEGLEDwgIKEC4YgAQYQxiKBcICERAAGIAEGJECGLEDGIMBGIoFwgIEEAAYA8ICBRAhGKABmAMAiAYBkAYIkgcEMjEuMaAHl26yBwQxNi4xuAfmC8IHBjItMTYuNsgHhwE&sclient=gws-wiz-serp)__ on the difference between US numbered highways and US interstate highway system

__[More granular website](https://99percentinvisible.org/article/beyond-streets-avenues-simple-visual-guide-different-types-roads/)__


Federal Roads.  Highways & Interstates

In [ ]:
highways = streets.str.startswith('US').sum()+ streets.str.startswith('United States').sum()

interstates = streets.str.startswith('I-').sum() + streets.str.startswith('Interstate').sum()

fed_roads = highways + interstates

msg = f'There are roughly {fed_roads} incidents which occured on a federal roads:  '
msg += f'{highways} on a numbered highways and {interstates} interstates.'

print(msg)

In [ ]:
highways = streets[streets.str.contains('Highway',na=False)]
highways.value_counts(dropna=False,normalize=True).sort_values(ascending=False)

Routes

In [ ]:
routes = streets[streets.str.startswith('Rt',na=False) | streets.str.startswith('Route',na=False)]
print(routes.shape[0])
routes.value_counts(dropna=False,normalize=True).sort_values(ascending=False)


In [ ]:
routes = streets.str.startswith('Route').sum() + streets.str.startswith('Rt').sum()
routes

### Weather

#### Quantitative Features

In [ ]:
weather_factors = ['Temperature(F)','Wind_Chill(F)',
                   'Humidity(%)','Pressure(in)',
                   'Visibility(mi)','Wind_Speed(mph)',
                   'Precipitation(in)']

data[weather_factors].describe()

#### Categorical Features

In [ ]:
weather_descriptions = data['Weather_Condition'].unique()
print(f'There are {weather_descriptions.shape[0]} ways that the weather is described in this dataset: \n')
print(weather_descriptions)

In [ ]:
weather_desc_counts = data['Weather_Condition'].value_counts(dropna=False,normalize=True)
weather_desc_counts.sort_values(ascending=False).head(25).plot()

This weather condition feature is too granular to be of any analytical value.  We need to consider consolidating this feature or ignore it all together.  May be of interest to consider how these categories relate to the numeric weather features. 

### Night/Day Features

There appears to be multiple indicators for day/night that need to be explored.  According to __[Google](https://www.google.com/search?q=different+types+of+twilight&rlz=1C5OZZY_enUS1127US1127&oq=different+types+of+twilight&gs_lcrp=EgZjaHJvbWUyCQgAEEUYORiABDIICAEQABgWGB4yCAgCEAAYFhgeMggIAxAAGBYYHjIICAQQABgWGB4yDQgFEAAYhgMYgAQYigUyDQgGEAAYhgMYgAQYigUyDQgHEAAYhgMYgAQYigUyDQgIEAAYhgMYgAQYigUyCggJEAAYgAQYogTSAQg0NjU3ajFqN6gCALACAA&sourceid=chrome&ie=UTF-8)__, Nautical, Astronomical and Civil all have precise definitions--based on usage--that do not, to me, inform as to how their distinctions contribute to understanding accident patterns.  And as you can see below, there is no clear pattern in how they differ from one another in classifying day vs night.

Ultimately, I think dropping all of them except for Sunrise_Sunset is for the best.  

### Night/Day Features

There appears to be multiple indicators for day/night that need to be explored.  According to __[Google](https://www.google.com/search?q=different+types+of+twilight&rlz=1C5OZZY_enUS1127US1127&oq=different+types+of+twilight&gs_lcrp=EgZjaHJvbWUyCQgAEEUYORiABDIICAEQABgWGB4yCAgCEAAYFhgeMggIAxAAGBYYHjIICAQQABgWGB4yDQgFEAAYhgMYgAQYigUyDQgGEAAYhgMYgAQYigUyDQgHEAAYhgMYgAQYigUyDQgIEAAYhgMYgAQYigUyCggJEAAYgAQYogTSAQg0NjU3ajFqN6gCALACAA&sourceid=chrome&ie=UTF-8)__, Nautical, Astronomical and Civil all have precise definitions--based on usage--that do not, to me, inform as to how their distinctions contribute to understanding accident patterns.  

In [ ]:
print(pd.crosstab(data['Nautical_Twilight'],data['Astronomical_Twilight']))
print('-'*50)
print(pd.crosstab(data['Nautical_Twilight'],data['Civil_Twilight']))
print('-'*50)
print(pd.crosstab(data['Nautical_Twilight'],data['Sunrise_Sunset']))

In [ ]:
print(pd.crosstab(data['Astronomical_Twilight'],data['Civil_Twilight']))
print('-'*50)
print(pd.crosstab(data['Astronomical_Twilight'],data['Sunrise_Sunset']))

In [ ]:
print(pd.crosstab(data['Civil_Twilight'],data['Sunrise_Sunset']))

 And as you can see, there is no clear pattern in how they differ from one another in classifying day vs night.

Ultimately, I think dropping all of them except for Sunrise_Sunset is for the best.

ACTION DECISION: Remove Nautical_Twilight, Civil_Twilight, Astronomical_Twilight and leave Sunrise_Sunse.

In [ ]:
_USELESS_.extend(['Nautical_Twilight','Civil_Twilight','Astronomical_Twilight'])

### Infrastructure
Twelve binary features indicating the existance of a traffic design or a PoI.

In [ ]:
infrastructures = ['Amenity','Bump','Crossing','Give_Way',
                   'Junction','No_Exit','Railway','Roundabout',
                   'Station','Stop','Traffic_Calming','Traffic_Signal',
                   'Turning_Loop'
                   ]

Need to verify what the definitions of each of these are...

In [ ]:
data[infrastructures].sum().sort_values(ascending=False)

How often do these overlap?

In [ ]:
data[infrastructures].sum(axis=1).value_counts()

In [ ]:
sns.heatmap(data[infrastructures].corr(),vmin=-1,vmax=1,cmap='coolwarm');

# 3-Data Processing

Start with a brand new dataset

In [ ]:
del data
if _SPARK_:
    spark.stop()
else:
    pass

In [ ]:
if _SPARK_:
    spark = SparkSession.builder.appName("Accident Data Project").getOrCreate()
    data_clean = spark.read.csv('US_Accidents_March23.csv',header=True,inferSchema=True)
else:
    if _LITE_SWITCH_:
        data_clean = pd.read_csv('US_Accidents_March23_sampled_500k.csv')
    else:
        data_clean = pd.read_csv('US_Accidents_March23.csv')
    

## Drop Unnecessary Columns
Based on prior exploratory work.

In [ ]:
print(len(_USELESS_))
_USELESS_

In [ ]:
data_clean = data_clean.drop(_USELESS_,axis=1)

## Missing Data

### End Coordinates

In [ ]:
data_clean['End_Lat'] = data_clean['End_Lat'].fillna(data_clean['Start_Lat'])
data_clean['End_Lng'] = data_clean['End_Lng'].fillna(data_clean['Start_Lng'])

### Weather

In [ ]:
drop_missing = ['Weather_Condition','Temperature(F)','Pressure(in)','Visibility(mi)','Humidity(%)']
data_clean = data_clean.dropna(subset = drop_missing)

In [ ]:
data_clean['Precipitation(in)'] = data_clean['Precipitation(in)'].fillna(0)
data_clean['Wind_Speed(mph)'] = data_clean['Wind_Speed(mph)'].fillna(0)
data_clean['Wind_Direction'] = data_clean['Wind_Direction'].fillna('CALM')

In [ ]:
data_clean['Wind_Chill(F)'] = data_clean['Wind_Chill(F)'].fillna(data_clean['Temperature(F)'])

### Description

### Address

## Outliers

In [ ]:
condition = data_clean['Distance(mi)'] > 100
data_clean = data_clean[condition]

## Feature Engineering

### Impact Duration

In [ ]:
if _SPARK_:
    pass
else:
    data_clean['Start'] = pd.to_datetime(data_clean['Start_Time'],format='mixed')
    data_clean['End'] = pd.to_datetime(data_clean['End_Time'],format='mixed')

And create a new target feature describing the nature of the accident as a function of time.

In [ ]:
data_clean['Time_of_Impact(hr)'] = (data_clean['End'] - data_clean['Start']).dt.seconds/3600

In [ ]:
data_clean['Time_of_Impact(hr)'].describe()

### Partition the time data

In [ ]:
if _SPARK_:
    pass
else:
    data_clean['Start'] = pd.to_datetime(data_clean['Start_Time'],format='mixed')
    data_clean['End'] = pd.to_datetime(data_clean['End_Time'],format='mixed')

In [ ]:
if _SPARK_:
    pass
else:
    data_clean['Month'] = data_clean['Start'].dt.month
    data_clean['Year'] = data_clean['Start'].dt.year
    data_clean['DayofYear'] = data_clean['Start'].dt.dayofyear
    data_clean['DayofMonth'] = data_clean['Start'].dt.day
    data_clean['DayofWeek'] = data_clean['Start'].dt.day_of_week
    data_clean['Quarter'] = data_clean['Start'].dt.quarter
    data_clean['Hour'] = data_clean['Start'].dt.hour
    data_clean['Date'] = data_clean['Start'].dt.date

### Explore these new features...

In [ ]:
data_clean['Year'].value_counts(normalize=True).sort_index()

Concering the book end years are so under represented.  Consider removing entirely?

In [ ]:
data_clean['Quarter'].value_counts(normalize=True)

In [ ]:
data_clean['Month'].value_counts(normalize=True).sort_index()

In [ ]:
data_clean['DayofWeek'].value_counts(normalize=True).sort_index()

In [ ]:
data_clean['Hour'].value_counts(normalize=True).sort_index()

### Weekend

In [ ]:
data_clean['Weekend'] = ((data_clean['DayofWeek'] == 0) | (data_clean['DayofWeek'] == 6))

In [ ]:
print(f'There are {data_clean['Weekend'].sum()} observations which occured over the weekend.')
data_clean.groupby('Weekend')['Severity'].mean()

### Holidays
Per __[Google](https://www.google.com/search?client=safari&rls=en&q=holidays+with+most+traffic&ie=UTF-8&oe=UTF-8)__, there are a number of holidays in the US with the highest amount of traffic.  (I subbed NYE in for XMas.)

In [ ]:
thanksgivings = [get_thanksgiving_date(y) for y in range(2016,2024)]
memorials = [calculate_memorial_day(y) for y in range(2016,2024)]
labors = [get_labor_day(y) for y in range(2016,2024)]

In [ ]:
july_4 = ((data_clean['Month'] == 7) & (data_clean['DayofMonth'] == 4))
thanksgiving = data_clean['Date'].apply(lambda d:d in thanksgivings)
memorial = data_clean['Date'].apply(lambda d:d in memorials)
labor = data_clean['Date'].apply(lambda d:d in labors)
nye = ((data_clean['Month'] == 12) & (data_clean['DayofMonth'] == 31))

In [ ]:
holidays = (july_4 | thanksgiving | memorial | labor | nye)
data_clean['Holiday'] = holidays

In [ ]:
info = f'''{holidays.sum()} incidents occured on a prime holiday.'''

print(info)

### Rush Hour Indicator

__[Per Google:  ](https://www.google.com/search?client=safari&rls=en&q=rush+hour+typically&ie=UTF-8&oe=UTF-8)__ The morning rush hour begins around 6a, peaking between 7a & 9a, and eases off by 10a.  While afternoon/evening rush hour begins around 3p, peaking between 4p and 6p, and eases off by 7p.

In [ ]:
data_clean['Morning_Rush'] = ((6 > data_clean['DayofWeek']) & (data_clean['DayofWeek'] > 0) & (data_clean['Start'].dt.hour >= 6) & (10 >= data_clean['Start'].dt.hour))

In [ ]:
print(f'Total of {data_clean['Morning_Rush'].sum()} incidents during the morning rush hour')

In [ ]:
data_clean['Evening_Rush'] = ((6 > data_clean['DayofWeek']) & (data_clean['DayofWeek'] > 0) & (data_clean['Start'].dt.hour >= 15) & (19 >= data_clean['Start'].dt.hour))

In [ ]:
print(f'Total of {data_clean['Evening_Rush'].sum()} incidents during the evening rush hour')

In [ ]:
data_clean['Rush_Hour'] = data_clean['Evening_Rush'] | data_clean['Morning_Rush']

### Attach Location-Based Data to DF

Resources

https://www.faa.gov/air_traffic/publications/atpubs/cnt_html/appendix_a.html


https://apps.bea.gov/regional/geography.htm

https://en.wikipedia.org/wiki/List_of_regions_of_the_United_States

https://www2.census.gov/geo/pdfs/reference/GARM/Ch6GARM.pdf


https://en.wikipedia.org/wiki/Megaregions_of_the_United_States

### Consider Street Type Feature(s)

## Saveout Data

In [ ]:
if _LITE_SWITCH_:
    data_clean.to_csv('AccidentData_Sampled_Clean.csv')
else:
    data_clean.to_csv('AccidentData_Clean.csv')

In [ ]:
del data_clean
if _SPARK_:
    spark.stop()
else:
    pass

# Full EDA
Now with cleaned up data with additional features, there are still some areas of exploration that I want to cover to help inform A) possible other features and B) the statistical analyses.

In [ ]:
if _SPARK_:
    spark = SparkSession.builder.appName("Accident Data Project").getOrCreate()
    data = spark.read.csv('AccidentData_Clean.csv',header=True,inferSchema=True)
else:
    if _LITE_SWITCH_:
        data = pd.read_csv('AccidentData_Sampled_Clean.csv')
    else:
        data = pd.read_csv('AccidentData_Clean.csv')
    

In [ ]:
if _SPARK_:
    data.printSchema()
    print("Features: ",len(data.columns))
    print("Entries:  ",data.count())
else:
    data.info()

In [ ]:
_TARGET_ = ['Severity']
_NUMERICS_ = ['Distance(mi)','Time_of_Impact(hr)','Temperature(F)',
              'Wind_Chill(F)','Humidity(%)','Pressure(in)',
              'Visibility(mi)','Wind_Speed(mph)','Precipitation(in)']

## Nature of Incident Variables

## Relationships between Continous Variables

In [ ]:
if _SPARK_:
    corr_data = data.select(_NUMERICS_+['Severity'])
    col_names = corr_data.columns
    features = corr_data.rdd.map(lambda row: row[0:]) 
    corrs = Statistics.corr(features, method="pearson")
else:
    corrs = data[_NUMERICS_].corr()
    print(corrs)
   

In [ ]:
sns.heatmap(corrs,vmin=-1,vmax=1,cmap='coolwarm');

Comments

## Dates

In [ ]:
if _SPARK_:
    accdidents_year = data.stat.crosstab('Year','Severity')
else:
    accdidents_year = pd.crosstab(data['Year'],data['Severity'])

    accdidents_year['Total']=accdidents_year.sum(axis=1)
    accdidents_year['Average']=accdidents_year.apply(lambda r:sum(i*r[i] for i in range(1,5))/r['Total'],axis=1)
    accidents_calendar = pd.pivot_table(data,columns='Year',index='Month',values='Severity')

In [ ]:
display(accdidents_year)
print('*'*75)
display(accidents_calendar)

Comments & Concerns

Other Cross Tabs

### Infrastructure

In [ ]:
infrastructure_count = pd.DataFrame(columns=infrastructures)

### Weather Conditions

In [ ]:
weather_descriptions = data.Weather_Condition.unique()
print(f'There are {weather_descriptions.shape[0]} ways that the weather is described in this dataset: \n')
print(weather_descriptions)

In [ ]:
description_counts = data.Weather_Condition.value_counts(dropna=False,normalize=True)
description_counts.sort_values(ascending=False)

To use this feature, we need to consolidate somehow.  But first...

### Exploration of Weather Condition Consolidation

Is there a way, we can use the continous features to create a consolidated but meaningful feature which maybe can improve on the categorical weather feature. 

In [ ]:
weather_factors = ['Temperature(F)','Wind_Chill(F)','Humidity(%)',
                   'Pressure(in)','Visibility(mi)', 'Wind_Direction',
                   'Wind_Speed(mph)','Precipitation(in)']

#### PCA

In [ ]:
X = data[weather_factors].dropna()
scaler = StandardScaler() 
X_std =scaler.fit_transform(X)

In [ ]:
pca = PCA()
pca.fit(X_std);

In [ ]:
cumulative_var = np.cumsum(pca.explained_variance_ratio_)
# Create a cumulative variance plot
plt.figure(figsize=(10, 6))
cum_var_plot = plt.plot(range(1, len(cumulative_var) + 1), cumulative_var, 
         'o-', linewidth=2, color='green')
# Add lines for 90% and 95% thresholds
plt.axhline(y=0.9, color='r', linestyle='--', label='90% threshold')
plt.axhline(y=0.95, color='g', linestyle='--', label='95% threshold')
plt.title('Cumulative Variance Explained')
plt.xlabel('Number of Components')
plt.ylabel('Cumulative Variance Explained')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
target_variance = 0.95
n_components_variance = np.argmax(cumulative_var >= target_variance) + 1
print(f'We choose to include the first {n_components_variance} components.')

In [ ]:
pca = PCA(n_components=n_components_variance)
pca.fit(X_std);
# Transform the standardized data to get principal components
X_pca = pca.transform(X_std)

# Create a DataFrame with the principal components
pca_df = pd.DataFrame(
    data=X_pca,
    columns=[f'PC{i+1}' for i in range(X_pca.shape[1])]
)
pca_df

In [ ]:
# Create an easy to view DF of the loadings.
loadings = pd.DataFrame(
    pca.components_.T,
    columns=[f'PC{i+1}' for i in range(n_components_variance)],
    index=weather_factors
)
loadings

In [ ]:
# Visualize the loadings for the first two PCs
plt.figure(figsize=(12, 8))
for i, feature in enumerate(weather_factors):
    plt.arrow(0, 0, loadings.iloc[i, 0], loadings.iloc[i, 1], head_width=0.05, head_length=0.05)
    plt.text(loadings.iloc[i, 0]*1.1, loadings.iloc[i, 1]*1.1, feature, fontsize=12)

# Add a unit circle for reference
circle = plt.Circle((0, 0), 1, fill=False, linestyle='--')
plt.gca().add_patch(circle)

plt.grid(True)
plt.axhline(y=0, color='k', linestyle='-', alpha=0.3)
plt.axvline(x=0, color='k', linestyle='-', alpha=0.3)
plt.xlim(-1.1, 1.1)
plt.ylim(-1.1, 1.1)
plt.title('PCA Loading Plot (PC1 vs PC2)')
plt.xlabel(f'PC1 ({pca.explained_variance_ratio_[0]:.2%} variance explained)')
plt.ylabel(f'PC2 ({pca.explained_variance_ratio_[1]:.2%} variance explained)')
plt.tight_layout()
plt.show()

#### K-Means Clustering

In [ ]:
X = data[weather_factors+['Weather_Condition']].dropna()[weather_factors]
scaler = StandardScaler() 
X_std =scaler.fit_transform(X)

In [ ]:

km = KMeans(random_state=42)
visualizer = KElbowVisualizer(km,k=(5,50))
visualizer.fit(X_std)
visualizer.show();

Recall, the heavily-skewed distribution of weather descripitions.  

In [ ]:
km = KMeans(random_state=42,n_clusters=visualizer.elbow_value_)
km.fit(X_std)

In [ ]:
centroids = pd.DataFrame(scaler.inverse_transform(km.cluster_centers_),columns=weather_factors)
centroids

In [ ]:
pd.Series(km.labels_).value_counts(normalize=True).sort_values(ascending=False)

In [ ]:
described_obsv = pd.DataFrame(data[weather_factors+['Weather_Condition']].dropna()['Weather_Condition'])
described_obsv['Cluster'] = km.labels_
described_obsv['Severity'] = data['Severity']
described_obsv['Distance(mi)'] = data['Distance(mi)']
described_obsv

In [ ]:
cluster_v_descr = pd.crosstab(described_obsv['Cluster'],described_obsv['Weather_Condition'])
cluster_v_descr

In [ ]:
cluster_v_sever = pd.crosstab(described_obsv['Cluster'],described_obsv['Severity'])
cluster_v_sever['Average'] = cluster_v_sever.apply(lambda r:sum(r[c]*c for c in range(1,5))/sum(r),axis=1)
cluster_v_sever

#### Decision

### Final Saveout

In [ ]:
if _LITE_SWITCH_:
    data_clean.to_csv('AccidentData_Sampled_Clean_2.csv')
else:
    data_clean.to_csv('AccidentData_Clean_2.csv')

In [ ]:
del data_clean
if _SPARK_:
    spark.stop()
else:
    pass

# Statistical Analyses

Description of the three primary tests that are going to be run for all of these categorical features...

In [ ]:
if _SPARK_:
    spark = SparkSession.builder.appName("Accident Data Project").getOrCreate()
    data = spark.read.csv('AccidentData_Clean_2.csv',header=True,inferSchema=True)
else:
    if _LITE_SWITCH_:
        data = pd.read_csv('AccidentData_Sampled_Clean_2.csv')
    else:
        data = pd.read_csv('AccidentData_Clean_2.csv')
    

## Nature-of-Accident Features vs. Each Other

In [ ]:
model_distVseverity = ols("Q('Distance(mi)') ~ C(Severity)", data=data).fit()
anova_table_distVSeverity = sm.stats.anova_lm(model_distVseverity, typ=2)
print("\nANOVA for Distance vs Severity Category:\n", anova_table_distVSeverity)
p_val = anova_table_distVSeverity['PR(>F)'].iloc[0]
print(f"P-value for Distance vs Severity Category: {p_val}")

In [ ]:
model_timeVseverity = ols("Q('Time_of_Impact(hr)') ~ C(Severity)", data=data).fit()
anova_table_timeVseverity = sm.stats.anova_lm(model_timeVseverity, typ=2)
print("\nANOVA for Time of Impact vs Severity Category:\n", anova_table_timeVseverity)
p_val = anova_table_timeVseverity['PR(>F)'].iloc[0]
print(f"P-value for Time of Impact vs Severity Category: {p_val}")

Comments

In [ ]:
del model_distVseverity, model_timeVseverity
del anova_table_distVSeverity, anova_table_timeVseverity
del p_val

## Weather-Related


In [ ]:
if _SPARK_:
    spark = SparkSession.builder.appName("Accident Data Project").getOrCreate()
    data = spark.read.csv('AccidentData_Clean_2.csv',header=True,inferSchema=True)
else:
    if _LITE_SWITCH_:
        data = pd.read_csv('AccidentData_Sampled_Clean_2.csv')
    else:
        data = pd.read_csv('AccidentData_Clean_2.csv',usecols = ['Weather_Condition','Distance(mi)','Time_of_Impact(hr)'])
    

## Calendar-Related

In [ ]:
if _SPARK_:
    spark = SparkSession.builder.appName("Accident Data Project").getOrCreate()
    data = spark.read.csv('AccidentData_Clean_2.csv',header=True,inferSchema=True)
else:
    if _LITE_SWITCH_:
        data = pd.read_csv('AccidentData_Sampled_Clean_2.csv')
    else:
        tgt_cols = ['Month','Quarter','Year','DayofWeek','Weekend','Holiday']
        data = pd.read_csv('AccidentData_Clean_2.csv',usecols = tgt_cols+['Severity','Distance(mi)','Time_of_Impact(hr)'])
    

### YoY Difference?

In [ ]:
reports,stats = test_results('Year',data)
data.groupby('Year')[['Severity','Time_of_Impact(hr)','Distance(mi)']].mean()

### Quarterly Difference?

In [ ]:
reports,stats = test_results('Quarter',data)
data.groupby('Quarter')[['Severity','Time_of_Impact(hr)','Distance(mi)']].mean()

### Monthly ?

In [ ]:
reports,stats = test_results('Month',data)
data.groupby('Month')[['Severity','Time_of_Impact(hr)','Distance(mi)']].mean()

### Day of Week?

In [ ]:
reports,stats = test_results('DayofWeek',data)
data.groupby('DayofWeek')[['Severity','Time_of_Impact(hr)','Distance(mi)']].mean()

### Weekends?

In [ ]:
reports,stats = test_results('Weekend',data)
data.groupby('Weekend')[['Severity','Time_of_Impact(hr)','Distance(mi)']].mean()

### Holidays?

In [ ]:
reports,stats = test_results('Holiday',data)
data.groupby('Holiday')[['Severity','Time_of_Impact(hr)','Distance(mi)']].mean()

## Time of Day?

In [ ]:
if _SPARK_:
    spark = SparkSession.builder.appName("Accident Data Project").getOrCreate()
    data = spark.read.csv('AccidentData_Clean.csv',header=True,inferSchema=True)
else:
    if _LITE_SWITCH_:
        data = pd.read_csv('AccidentData_Sampled_Clean.csv')
    else:
        tgt_cols = ['Hour','Morning_Rush','Evening_Rush','Rush_Hour','Sunrise_Sunset']
        data = pd.read_csv('AccidentData_Clean.csv',usecols = tgt_cols+['Severity','Distance(mi)','Time_of_Impact(hr)'])
    

### Hourly?

In [ ]:
reports,stats = test_results('Hour',data)
data.groupby('Hour')[['Severity','Time_of_Impact(hr)','Distance(mi)']].mean()

Comments

### Rush Hours?


In [ ]:
reports,stats = test_results('Evening_Rush',data)
data.groupby('Evening_Rush')[['Severity','Time_of_Impact(hr)','Distance(mi)']].mean()

In [ ]:
reports,stats = test_results('Morning_Rush',data)
data.groupby('Morning_Rush')[['Severity','Time_of_Impact(hr)','Distance(mi)']].mean()

In [ ]:
reports,stats = test_results('Rush_Hour',data)
data.groupby('Rush_Hour')[['Severity','Time_of_Impact(hr)','Distance(mi)']].mean()

### Day/Night?

In [ ]:
reports,stats = test_results('Sunrise_Sunset',data)
data.groupby('Sunrise_Sunset')[['Severity','Time_of_Impact(hr)','Distance(mi)']].mean()

## Location-Related

In [ ]:
if _LITE_SWITCH_:
    pass
elif _SPARK_:
    pass
else:
    try:
        del data
        del reports, stats
    except:
        pass
    

In [ ]:
if _SPARK_:
    spark = SparkSession.builder.appName("Accident Data Project").getOrCreate()
    data = spark.read.csv('AccidentData_Clean.csv',header=True,inferSchema=True)
else:
    if _LITE_SWITCH_:
        data = pd.read_csv('AccidentData_Sampled_Clean.csv')
    else:
        tgt_cols = ['State']
        data = pd.read_csv('AccidentData_Clean.csv',usecols = tgt_cols + ['Severity','Distance(mi)','Time_of_Impact(hr)'])
    

### By State

In [ ]:
states = data['State'].unique()

for state in states:
    print(state + ':')
    data['inState'] = data['State'] == state
    test_results('inState',data,visuals=False)
    print('-'*75)

In [ ]:
data.groupby('State')[['Severity','Time_of_Impact(hr)','Distance(mi)']].mean()

### Infrastructure

In [ ]:
infrastructures = ['Amenity','Bump','Crossing','Give_Way',
                   'Junction','No_Exit','Railway','Roundabout',
                   'Station','Stop','Traffic_Calming','Traffic_Signal',
                   'Turning_Loop'
                   ]

In [ ]:
if _SPARK_:
    spark = SparkSession.builder.appName("Accident Data Project").getOrCreate()
    data = spark.read.csv('AccidentData_Clean.csv',header=True,inferSchema=True)
else:
    if _LITE_SWITCH_:
        data = pd.read_csv('AccidentData_Sampled_Clean.csv')
    else:
        data = pd.read_csv('AccidentData_Clean.csv',usecols = infrastructures + ['Severity','Distance(mi)','Time_of_Impact(hr)'])
    

In [ ]:
for structure in infrastructures:
    print(structure+": ")
    reports,stats = test_results(structure,data,visuals=True)
    display(data.groupby(structure)[['Severity','Time_of_Impact(hr)','Distance(mi)']].mean())
    print('-'*75)

# 6-Advanced Analysis (Wish List)

# 7-Insights & Conclusions

## Basic Insights

### Annual Distinctions

Number of Accidents; Distribution of Severities; Length of Impact

### Temporal & Spatial Considerations

How do accident counts relate to different times of the day and for different region types (urban vs rural)?

### Weather Considerations
Does certain weather conditions produce more severe incidents?  

# Advanced (Wish List) Analysis

## Unusual Weather 

Does driving in unexpected weather--based on area, time of year and/or both--create a higher likelihood of an accident.

## Famous Highways

## Highway's Near Urban Areas

For cross-state

## Naturual Language

Examination of the description feature.

## Safety Infrastructure
Does certain road infrastructure projects help reduce the number of incidents?

## New Traffic Pattern

Does the existence of a new traffic pattern in the area increase the likelihood of an accident?

## Recent Accident Indicator

Does the presence of one accident, predict another.